In [1]:
import json
import datetime
import random
import time
import string
import argparse
import os
import tensorflow as tf
import numpy as np
from tensorflow.models.rnn import rnn_cell
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops

random.seed(0)
np.random.seed(0)

from utils import train_utils
from utils import googlenet_load

import xml
import numpy as np
from pylab import *
%matplotlib inline
import xml.etree.ElementTree as ET
import cv2
import os
from matplotlib import patches
from random import shuffle

In [2]:
import numpy as np
import random
import json
import os
import cv2
from scipy.misc import imread, imresize

from utils.data_utils import (annotation_jitter, image_to_h5, get_cell_grid)
from utils.train_utils import add_rectangles
from utils.annolist import AnnotationLib as al
from utils.rect import Rect
import copy

In [3]:
# This system might perform poorly if the GoogleNet model was trained on data different than we're feeding.
# In particular, different methods of cropping or resizing, BGR vs RGB, different whitening, etc could have 
# very large effects on how well this system works. 

In [4]:
H = {
    "data": {
        "train_idl": "./data/pascal/pascal_voc_train.idl",
        "test_idl": "./data/pascal/pascal_voc_train.idl",
        "truncate_data": False   # Only loads first 10 files
    },
    "arch": {
        "use_lstm": False,
        "use_dropout": True, 
        "image_width": 640, 
        "image_height": 480,
        "grid_height": 15, 
        "grid_width": 20,
        "batch_size": 2, 
        "region_size": 32,
        "lstm_size": 200,
        "num_classes": 21,
        "rnn_len": 1
    }, 
    "logging": {
    },
    "solver": {
        "opt": "RMS",
        "use_jitter": False,
        "rnd_seed": 1,
        "epsilon": 0.00001, 
        "learning_rate": 0.001, 
        "learning_rate_step": 7500, 
        "weights": "",
        "head_weights": [1.0, 0.1]   # The relative weighting of loss for bounding boxes and class predictions (not sure order)
    }
}
H['solver']['gpu'] = 0
H['exp_name'] = "pascal"
H['save_dir'] = "/home/ubuntu/external_drive/tensorbox_outputs" + '/%s_%s' % (H['exp_name'],
        datetime.datetime.now().strftime('%Y_%m_%d_%H.%M'))
H['arch']['num_classes'] = 2

In [5]:
# Makes a Rect object from my box class. Used to get the cool Rect methods. Confidence set to -1.
def box_to_rect(box):
    return Rect(box.xmin, box.ymin, box.xmax - box.xmin, box.ymax - box.ymin, confidence = -1)

def box_to_anno_rect(box):
    return al.AnnoRect(box.xmin, box.ymin, box.xmax, box.ymax)

def annotation_to_h5(a, cell_width, cell_height, max_len):
    region_size = 32
    # Makes a cell_width x cell_height grid of region_size x region_size rects
    cell_regions = get_cell_grid(cell_width, cell_height, region_size)

    cells_per_image = len(cell_regions)
    
    # Makes a blank list of boxes 
    box_list = [[] for idx in range(cells_per_image)]
            
    for cidx, c in enumerate(cell_regions):
        # put in box_list[cell_index] the list of all annotations that intersect somehow ??? with this box
        box_list[cidx] = [box_to_anno_rect(r) for r in a[1] if all(box_to_anno_rect(r).intersection(c))]
    
    # a list of cells in the image
    # [0, cell_id, :, box_id, 0] returns the box coords and size
    boxes = np.zeros((1, cells_per_image, 4, max_len, 1), dtype = np.float)
    
    # a flag to indicate if there's an item in the box
    box_flags = np.zeros((1, cells_per_image, 1, max_len, 1), dtype = np.float)
    
    # Iterate through each cell
    for cidx in range(cells_per_image):
        
        # Get the number of classes in this cell. Since max_len is default 1, ...
        cur_num_boxes = min(len(box_list[cidx]), max_len)
        #assert(cur_num_boxes <= max_len)
        
        # Indicate that the first cur_num_boxes rows of the array have an associated box 
        box_flags[0, cidx, 0, 0:cur_num_boxes, 0] = 1
        
        # Get the coords of the center of the cell
        cell_ox = 0.5*(cell_regions[cidx].x1 + cell_regions[cidx].x2)
        cell_oy = 0.5*(cell_regions[cidx].y1 + cell_regions[cidx].y2)
        
        # create a list of the relative positions of annotation boxes within each cell
        unsorted_boxes = []
        for bidx in range(cur_num_boxes):

            # relative box position with respect to cell
            ox = 0.5 * (box_list[cidx][bidx].x1 + box_list[cidx][bidx].x2) - cell_ox
            oy = 0.5 * (box_list[cidx][bidx].y1 + box_list[cidx][bidx].y2) - cell_oy

            width = abs(box_list[cidx][bidx].x2 - box_list[cidx][bidx].x1)
            height= abs(box_list[cidx][bidx].y2 - box_list[cidx][bidx].y1)

            unsorted_boxes.append(np.array([ox, oy, width, height], dtype=np.float))
        
        # Sort boxes by distance from center of cell
        for bidx, box in enumerate(sorted(unsorted_boxes, key=lambda x: x[0]**2 + x[1]**2)):
            boxes[0, cidx, :, bidx, 0] = box
    
    # Returns data containing the relative positions of the annotation boxes within each cell in a grid
    # and a flag indicating which cells actually have annotations within them
    # boxes[0, cell_id, :, box_id, 0] returns the box coords and size
    # box_flags[0, cell_id, 0, box_id, 0] returns whether or not there is actually a box at the given indices
    return boxes, box_flags

# Rescales an image and its bounding boxes to the target dimensions
def rescale_boxes(I, anno, target_height, target_width):
    x_scale = target_width / float(I.shape[1])
    y_scale = target_height / float(I.shape[0])
    this_anno = copy.deepcopy(anno)
    for r in this_anno[1]:
        #r = box_to_rect(r)
        assert r.xmin < r.xmax
        r.xmin *= x_scale
        r.xmax *= x_scale
        assert r.ymin < r.ymax
        r.ymin *= y_scale
        r.ymax *= y_scale
    I_r = imresize(I, (target_height, target_width), interp='cubic')
    return I_r, this_anno

def load_idl_tf(H, jitter):
    """Take the idlfile and net configuration and create a generator
    that outputs a jittered version of a random image from the annolist
    that is mean corrected."""

    # Pull annotations from the global "annotations"
    # annotations is a list of (filename, bounding_boxes)'s
    annos = annotations
    random.seed(0)
    # Only use first 10 if truncate_data flag is set
    #if H['data']['truncate_data']:
    #    annos = annos[:10]
    while True:
        random.shuffle(annos)
        for anno in annos:
            # Load in the image contained in the filename
            I = imread(anno[0])
            if I.shape[0] != H["arch"]["image_height"] or I.shape[1] != H["arch"]["image_width"]:
                I, anno = rescale_boxes(I, anno, H["arch"]["image_height"], H["arch"]["image_width"])
            """
            if jitter:
                jitter_scale_min=0.9
                jitter_scale_max=1.1
                jitter_offset=16
                I, anno = annotation_jitter(I,
                                            anno, target_width=H["arch"]["image_width"],
                                            target_height=H["arch"]["image_height"],
                                            jitter_scale_min=jitter_scale_min,
                                            jitter_scale_max=jitter_scale_max,
                                            jitter_offset=jitter_offset)
            """
            # Idk what this shit is, and it'll definitely break..
            boxes, flags = annotation_to_h5(anno,
                                            H["arch"]["grid_width"],
                                            H["arch"]["grid_height"],
                                            H["arch"]["rnn_len"])

            yield {"image": I, "boxes": boxes, "flags": flags}

In [6]:
# Test function for input generator
def do_input_test(generator):
    
    data = next(generator)
    image = data["image"]
    flags = data["flags"]
    boxes = data["boxes"]
    #plt.imshow(image)
    image = image.reshape([1, 480, 640, 3])
    boxes = boxes.reshape([300, 4])
    flags = flags.reshape([300, 1])
    confs = np.array([[make_sparse(detection, d=2) for detection in cell] for cell in flags])
    confs = confs.reshape([300, 2])
    # This probably overlays the rectange on the image???
    test_output_to_log = train_utils.add_rectangles(image,
                                                    confs,
                                                    boxes,
                                                    H["arch"])[0]
    #print(boxes)
    plt.figure()
    plt.imshow(test_output_to_log)

In [9]:
# Creates a 1-hot encoding of n given total num classes d
def make_sparse(n, d):
    v = np.zeros((d,), dtype=np.float32)
    v[n] = 1.
    return v

def load_data_gen(H, phase, jitter):
    grid_size = H["arch"]['grid_width'] * H["arch"]['grid_height']

    data = load_idl_tf(H, jitter={'train': jitter, 'test': False}[phase])

    for d in data:
        output = {}
        
        rnn_len = H["arch"]["rnn_len"]
        flags = d['flags'][0,:,0,0:rnn_len,0]
        boxes = np.transpose(d['boxes'][0,:,:,0:rnn_len,0], (0,2,1))
        assert(flags.shape == (grid_size, rnn_len))
        assert(boxes.shape == (grid_size, rnn_len, 4))

        output['image'] = d['image']
        output['confs'] = np.array([[make_sparse(detection, d=2) for detection in cell] for cell in flags])
        output['boxes'] = boxes
        output['flags'] = flags
        
        yield output


In [ ]:
"""
How this network works:

Architecture:
    Standard conv-net. Final convolutional layer is connected to two fully-connected layers (in parallel, not series).
    Confidences layer outputs a [num_classes] vector with predicted class confdiences for each position in the localization grid.
    Bounding-box regressor layer outputs a regressed bounding box for each position in the grid.

Training:
    For a given training image, the localization grid is generated. For each position in the grid, a sparse vector 
    (like one-hot, but with a few places that can be "hot") is created that defines if a certain class is in that box in the image.
    Also, for each occurance of a class, a "regression target" is created of where the object actually is within the grid box.

"""

In [11]:
@ops.RegisterGradient("Hungarian")
def _hungarian_grad(op, *args):
    return map(array_ops.zeros_like, op.inputs)

In [12]:
def build_overfeat_forward(H, x, googlenet, phase):
    """
    Inputs:
        H: hyperparameters
        x: inputs in some form???
        googlenet: an initialized googlenet instance
        phase: {train, test}
    Returns:
        pred_boxes: the op that predicts bounding boxes
        pred_logits: the linear op that predicts classes
        pred_confidences: the softmax class classifier op. Size [num_locations * batch_size, num_classes]
    """
    # Subtract image mean from image
    input_mean = 117.
    x -= input_mean
    
    # Grab the last convolutional layer from a new GoogleNet graph
    Z = googlenet_load.model(x, googlenet, H)
    
    grid_size = H['arch']['grid_width'] * H['arch']['grid_height']
    
    # If necessary, add a dropout layer to the graph
    if H['arch']['use_dropout'] and phase == 'train':
        Z = tf.nn.dropout(Z, 0.5)
    
    # Currently, W[0] is [1024, num_classes], and B is [num_classes]
    # Z is [batch_size * grid_width * grid_height, 1024]
    pred_logits = tf.matmul(Z, googlenet['W'][0]) + googlenet['B'][0] #, name=phase+'/logits_0')
    # Reshape to [batch_size * grid_width * grid_height, num_classes]
    # I think it's already this shape???
    pred_logits = tf.reshape(pred_logits, [H['arch']['batch_size'] * grid_size, H['arch']['num_classes']])
    # Do a softmax on this
    # Result is, for each location, the confidence of each class
    pred_confidences = tf.nn.softmax(pred_logits)
    
    # Now, do the same thing for the boxes
    # WHAT THE HELL IS THE * 100 at the end???
    # also, these reshapes make no sense - it's just un-reshaped in build_overfeat...
    pred_boxes = tf.reshape(tf.nn.xw_plus_b(Z, googlenet['W'][1], googlenet['B'][1],
                                            name=phase+'/logits_1'), 
                            [H['arch']['batch_size'] * grid_size, 1, 4]) * 100
    return pred_boxes, pred_logits, pred_confidences


def build_overfeat(H, x, googlenet, phase, boxes, confidences_r):
    # Build the forward part
    pred_boxes, pred_logits, pred_confidences = build_overfeat_forward(H, x, googlenet, phase)

    grid_size = H['arch']['grid_width'] * H['arch']['grid_height']
    
    # Reshape boxes to [num_locations * batch_size, 4]
    boxes = tf.cast(tf.reshape(boxes, [H['arch']['batch_size'] * grid_size, 4]), 'float32')
    
    # Calculate cross-entropy between real classes and predictions at each grid position
    cross_entropy = -tf.reduce_sum(confidences_r*tf.log(tf.nn.softmax(pred_logits) + 1e-6))
    
    # Loss function based on difference between predicted boxes and actual boxes for boxes with an object
    box_regressor_loss = tf.abs(pred_boxes[:, 0, :] - boxes) * tf.expand_dims(confidences_r[:, 1], 1)
    
    # Apply weights to losses 
    L = (H['solver']['head_weights'][0] * cross_entropy,
         H['solver']['head_weights'][1] * box_regressor_loss)
    
    # Sum up weighted losses for confidences and normalize to batch and grid size
    confidences_loss = (tf.reduce_sum(L[0], name=phase+'/confidences_loss') /
                        (H['arch']['batch_size'] * grid_size))
    
    # Sum up weighted losses for boxes and normalize to batch and grid size
    boxes_loss = (tf.reduce_sum(L[1], name=phase+'/boxes_loss') /
                  (H['arch']['batch_size'] * grid_size))
    
    # Total loss
    loss = confidences_loss + boxes_loss
    
    return pred_boxes, pred_confidences, loss, confidences_loss, boxes_loss

def build(H, q):
    '''
    Build full model for training, including forward / backward passes,
    optimizers, and summary statistics.
    Inputs:
        H: hyperparameter dict
        q: dict with queues for train and test input examples
    '''
    arch = H['arch']
    solver = H["solver"]

    os.environ['CUDA_VISIBLE_DEVICES'] = str(solver['gpu'])

    #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
    gpu_options = tf.GPUOptions()
    config = tf.ConfigProto(gpu_options=gpu_options)
    
    # Run googlenet_load init function
    googlenet = googlenet_load.init(H, config)
    
    # Configure solver
    learning_rate = tf.placeholder(tf.float32)
    if solver['opt'] == 'RMS':
        opt = tf.train.RMSPropOptimizer(learning_rate=learning_rate,
                                        decay=0.9, epsilon=solver['epsilon'])
    elif solver['opt'] == 'SGD':
        opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    else:
        raise ValueError('Unrecognized opt (optimizer) type')
    
    # We build two separate networks for training and testing!!! (is this necessary???)
    loss, accuracy, confidences_loss, boxes_loss = {}, {}, {}, {}
    for phase in ['train', 'test']:
        # Get inputs from queue
        x, confidences, boxes = q[phase].dequeue_many(arch['batch_size'])
        
        # ???
        flags = tf.argmax(confidences, 3)


        grid_size = H['arch']['grid_width'] * H['arch']['grid_height']
        confidences_r = tf.cast(
            tf.reshape(confidences[:, :, 0, :],
                       [H['arch']['batch_size'] * grid_size, arch['num_classes']]), 'float32')
        
        (pred_boxes, pred_confidences,
         loss[phase], confidences_loss[phase],
         boxes_loss[phase]) = build_overfeat(H, x, googlenet, phase, boxes, confidences_r)


        # Set up summary operations for tensorboard
        a = tf.equal(tf.argmax(confidences_r, 1), tf.argmax(pred_confidences, 1))
        accuracy[phase] = tf.reduce_mean(tf.cast(a, 'float32'), name=phase+'/accuracy')

        if phase == 'train':
            global_step = tf.Variable(0, trainable=False)
            train_op = opt.minimize(loss['train'], global_step=global_step)
        elif phase == 'test':
            test_image = x
            moving_avg = tf.train.ExponentialMovingAverage(0.99)
            smooth_op = moving_avg.apply([accuracy['train'], accuracy['test'],
                                          confidences_loss['train'], boxes_loss['train'],
                                          confidences_loss['test'], boxes_loss['test'],
                                          ])

            for p in ['train', 'test']:
                tf.scalar_summary('%s/accuracy' % p, accuracy[p])
                tf.scalar_summary('%s/accuracy/smooth' % p, moving_avg.average(accuracy[p]))
                tf.scalar_summary("%s/confidences_loss" % p, confidences_loss[p])
                tf.scalar_summary("%s/confidences_loss/smooth" % p,
                    moving_avg.average(confidences_loss[p]))
                tf.scalar_summary("%s/regression_loss" % p, boxes_loss[p])
                tf.scalar_summary("%s/regression_loss/smooth" % p,
                    moving_avg.average(boxes_loss[p]))

            # show ground truth to verify labels are correct
            test_true_confidences = confidences_r
            test_true_boxes = boxes[0, :, 0, :]

            # show predictions to visualize training progress
            test_pred_confidences = pred_confidences
            test_pred_boxes = pred_boxes[:, 0, :]

    summary_op = tf.merge_all_summaries()

    return (config, loss, accuracy, summary_op, train_op, googlenet['W_norm'],
            test_image, test_pred_boxes, test_pred_confidences,
            test_true_boxes, test_true_confidences, smooth_op,
            global_step, learning_rate)

In [13]:
# PASCAL VOC Data import
home_dir = os.path.expanduser('~') + "/"
datasets_dir = home_dir + "external_drive/"
voc_2012_dir = datasets_dir + "VOC/VOC2012/VOCdevkit/VOC2012/"

classes_to_index = {"aeroplane":1, "bicycle":2, "boat":3, "bottle":4, "bus":5, "car":6, "cat":7,
"chair":8, "cow":9, "diningtable":10, "dog":11, "horse":12, "motorbike":13, "person":14,
"pottedplant":15, "sheep":16, "train":17, "tvmonitor":18, "sofa":19, "bird":20}

indexes_to_classes = [0]*21
for key, value in classes_to_index.items():
    indexes_to_classes[value] = key

class bounding_box:
    def __init__(self, class_name, xmin, xmax, ymin, ymax):
        self.class_name = class_name
        #self.bbox_coords = bbox_coords #xmax, xmin, ymax, ymin
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax

# Get list of training images
train_filenames = []
with open(voc_2012_dir + "ImageSets/Main/train.txt", "r") as file:
    for line in file:
        train_filenames.append(line.strip())
        
# Read an annotation file, return a list of bounding boxes
def get_bounding_boxes(filename):
    bounding_boxes = []
    xmldoc = ET.parse(voc_2012_dir + "Annotations/{}.xml".format(filename))
    objects = xmldoc.findall('object')
    for thing in objects:
        name = thing.find('name').text
        box = thing.find('bndbox')
        xmax = int(box.find('xmax').text)
        xmin = int(box.find('xmin').text)
        ymax = int(box.find('ymax').text)
        ymin = int(box.find('ymin').text)
        bounding_boxes.append(bounding_box(name, xmin, xmax, ymin, ymax))
    return bounding_boxes

# Make a list of annotations in the format (filename, bounding_boxes)
annotations = []
for filename in train_filenames:
    full_filename = voc_2012_dir + "JPEGImages/{}.jpg".format(filename)
    bounding_boxes = get_bounding_boxes(filename)
    annotations.append((full_filename, bounding_boxes))

In [14]:
do_saves = True

In [15]:
if do_saves:
    # Make sure directory for saving model checkpoints and data exists
    if not os.path.exists(H['save_dir']): 
        os.makedirs(H['save_dir'])

    # Dump hyperparameters to file for reference
    ckpt_file = H['save_dir'] + '/save.ckpt'
    with open(H['save_dir'] + '/hypes.json', 'w') as f:
        json.dump(H, f, indent=4)

# Set up placeholder ops for inputs
x_in = tf.placeholder(tf.float32)
confs_in = tf.placeholder(tf.float32)
boxes_in = tf.placeholder(tf.float32)

# Queue to hold train and test inputs - (x_in, confs_in, boxes_in)
q = {}
# Enqueue op for train or test queues
enqueue_op = {}
for phase in ['train', 'test']:
    # Each queue holds 3 tensors - each of type tf.float32
    dtypes = [tf.float32, tf.float32, tf.float32]
    # Set up a (default 15x20 grid) to do ???
    grid_size = H['arch']['grid_width'] * H['arch']['grid_height']
    shapes = (
        [H['arch']['image_height'], H['arch']['image_width'], 3], # Size of input image
        [grid_size, H['arch']['rnn_len'], H['arch']['num_classes']], # ???
        [grid_size, H['arch']['rnn_len'], 4], # ???
        )
    q[phase] = tf.FIFOQueue(capacity=30, dtypes=dtypes, shapes=shapes)
    enqueue_op[phase] = q[phase].enqueue((x_in, confs_in, boxes_in))

# Make a dictionary to feed into the training step of tensorflow
def make_feed(d):
    return {x_in: d['image'], confs_in: d['confs'], boxes_in: d['boxes'],
            learning_rate: H['solver']['learning_rate']}

# A loop that enqueues data - to be run in its own thread
# Phase is {train, test}
# gen is a generator for examples to enqueue
def MyLoop(sess, enqueue_op, phase, gen):
    for d in gen:
        sess.run(enqueue_op[phase], feed_dict=make_feed(d))

# Build the network! Returns a bunch of useful ops
(config, loss, accuracy, summary_op, train_op, W_norm,
 test_image, test_pred_boxes, test_pred_confidences,
 test_true_boxes, test_true_confidences,
 smooth_op, global_step, learning_rate) = build(H, q)


# Create ops to log an input image, if given to fill placeholder test_image_to_log
test_image_to_log = tf.placeholder(tf.uint8,
                                   [H['arch']['image_height'], H['arch']['image_width'], 3])
log_image_name = tf.placeholder(tf.string)
log_image = tf.image_summary(log_image_name, tf.expand_dims(test_image_to_log, 0))

In [16]:
#derp

In [17]:
sess = tf.InteractiveSession()

In [18]:
# Set up a saver
saver = tf.train.Saver(max_to_keep=None)
writer = tf.train.SummaryWriter(
    logdir=H['save_dir'], 
    graph_def=sess.graph_def,
    flush_secs=10
)

In [19]:
#next(gen)

In [20]:
# Start threads to keep queues filled with input examples
threads = []
for phase in ['train', 'test']:
    # enqueue once manually to avoid thread start delay
    gen = load_data_gen(H, phase, jitter=H['solver']['use_jitter'])
    d = next(gen)
    sess.run(enqueue_op[phase], feed_dict=make_feed(d))
    
    # Create a thread to enqueue data
    threads.append(tf.train.threading.Thread(target=MyLoop,
                                             args=(sess, enqueue_op, phase, gen)))
    threads[-1].start()

# Set tensorflow random seed from hyperparameter config
tf.set_random_seed(H['solver']['rnd_seed'])

# Initialize variables
sess.run(tf.initialize_all_variables())

/home/ubuntu/venv/standard/lib/python3.4/site-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [21]:
# restore from checkpoint
path_to_checkpoint = "/home/ubuntu/packages/tensorbox/pascal_2016_03_13_06.40/save.ckpt-17000"
saver.restore(sess, path_to_checkpoint)

/home/ubuntu/venv/standard/lib/python3.4/site-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
# train model for N iterations
start = time.time()

for i in range(34000, 1000000):
    # Evaluate the network every display_iter iterations
    display_iter = 10
    
    # Learning rate decay function
    adjusted_lr = (H['solver']['learning_rate'] *
                   0.5 ** max(0, (i / H['solver']['learning_rate_step']) - 2))
    lr_feed = {learning_rate: adjusted_lr}
    
    if i % display_iter == 0:
        if i > 0:
            # Print timings
            dt = (time.time() - start) / (H['arch']['batch_size'] * display_iter)
        
        # Reset timer
        start = time.time()
        
        # Run network and get lots of things back
        (batch_loss_train, test_accuracy, weights_norm, 
            summary_str, np_test_image, np_test_pred_boxes,
            np_test_pred_confidences, np_test_true_boxes,
            np_test_true_confidences, _, _) = sess.run([
                 loss['train'], accuracy['test'], W_norm, 
                 summary_op, test_image, test_pred_boxes,
                 test_pred_confidences, test_true_boxes, test_true_confidences,
                 train_op, smooth_op,
                ], feed_dict=lr_feed)
        
        
        pred_true = [("%d_pred_output" % (i % 3), np_test_pred_boxes, np_test_pred_confidences),
                     ("%d_true_output" % (i % 3), np_test_true_boxes, np_test_true_confidences)]
        
        # For each box in the image (both real and predicted), create a summary image
        # with the box overlaid on it
        for name, boxes, confidences in pred_true:
            # This probably overlays the rectange on the image???
            test_output_to_log = train_utils.add_rectangles(np_test_image,
                                                            confidences,
                                                            boxes,
                                                            H["arch"])[0]
            
            # Check that test_output_to_log is indeed an image
            assert test_output_to_log.shape == (H['arch']['image_height'],
                                                H['arch']['image_width'], 3)
            
            # Feed the image into its placeholder
            feed = {test_image_to_log: test_output_to_log, log_image_name: name}
            
            # Get and add the image summary
            test_image_summary_str = sess.run(log_image, feed_dict=feed)
            if do_saves: writer.add_summary(test_image_summary_str, global_step=global_step.eval())
        
        # Add the real summary
        if do_saves: writer.add_summary(summary_str, global_step=global_step.eval())
        
        # Print progress update
        print_str = ', '.join([
            'Step: %d',
            'lr: %f',
            'Train Loss: %.2f',
            'Test Accuracy: %.1f%%',
            'Time/image (ms): %.1f'
        ])
        print(print_str % 
              (i, adjusted_lr, batch_loss_train,
               test_accuracy * 100, dt * 1000 if i > 0 else 0))
    else:
        # Run the training step
        batch_loss_train, _ = sess.run([loss['train'], train_op], feed_dict=lr_feed)
    
    # Every 1000 steps, save a checkpoint
    if global_step.eval() % 1000 == 0: 
        if do_saves: saver.save(sess, ckpt_file, global_step=global_step)

In [ ]:
do_pred = True
for _ in range(10):
    lr_feed = {learning_rate: .1}
    if do_pred:
        image, boxes, confidences = sess.run([test_image, test_pred_boxes, test_pred_confidences], feed_dict=lr_feed)
    else:
        image, boxes, confidences = sess.run([test_image, test_true_boxes, test_true_confidences], feed_dict=lr_feed)
    # This probably overlays the rectange on the image???
    test_output_to_log = train_utils.add_rectangles(image,
                                                    confidences,
                                                    boxes,
                                                    H["arch"])[0]
    #print(boxes)
    plt.figure()
    plt.imshow(test_output_to_log)
